# Extraction de Keywords

## Imports

In [2]:
import os
import yake
import requests
import collections
import string
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import sys
from nltk.corpus import stopwords
from IPython.display import Image
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from scipy.spatial.distance import cosine
from collections import Counter
from wordcloud import WordCloud
from collections import defaultdict
from spacy.lang.fr.examples import sentences
from bs4 import BeautifulSoup
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer



In [25]:
nltk.download('stopwords')
!python -m spacy download fr_core_news_md
nltk.download('punkt')
nltk.download("punkt_tab")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ---------------------------------------- 45.8/45.8 MB 6.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')


2024-12-13 18:23:32.466763: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-13 18:23:34.208025: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab 

True

## Extraire les mots clés d'un document avec Yake

In [4]:
kw_extractor = yake.KeywordExtractor(lan="fr", top=1000)
kw_extractor

In [5]:
# Lister les Fichiers
data_path = "../../data/txt/"         
files = [f for f in os.listdir(data_path) if f.endswith('.txt')]

In [6]:
# Imprimer le nombre de fichiers identifiés
len(files)

1001

In [7]:
files[:1000]

['BE-KBR00_15463334_19590401_22_00_00_0_01_0063_29132195.txt',
 'BE-KBR00_15463334_19611101_32_00_00_0_01_0035_29133524.txt',
 'BE-KBR00_15463334_19660101_49_00_00_0_01_0074_29142174.txt',
 'BE-KBR00_15463334_19691201_64_00_00_0_01_0016_29128000.txt',
 'BE-KBR00_15463334_19691201_64_00_00_0_01_0023_29128007.txt',
 'BE-KBR00_15463334_19700401_65_00_00_0_01_0051_29151819.txt',
 'BE-KBR00_15463334_19720301_73_00_00_0_01_0017_29572262.txt',
 'BE-KBR00_15463334_19720301_73_00_00_0_01_0019_29572264.txt',
 'BE-KBR00_15463334_19740701_83_00_00_0_01_0023_29580695.txt',
 'BE-KBR00_15463334_19781001_99_00_00_0_01_0039_29589982.txt',
 'BE-KBR00_15463334_19791101_103_00_00_0_01_0031_29592394.txt',
 'BE-KBR00_15463334_19820601_111_00_00_0_01_0035_29684278.txt',
 'BE-KBR00_15463334_19870601_127_00_00_0_01_0009_29693273.txt',
 'BE-KBR00_15463334_19871201_128_00_00_0_01_0047_29695113.txt',
 'KB_JB1051_1921-10-22_01-00004.txt',
 'KB_JB1051_1922-03-04_01-00004.txt',
 'KB_JB1051_1922-04-29_01-00004.txt',


In [9]:
# Choisir un fichier
this_file = files[0]
this_file

'BE-KBR00_15463334_19590401_22_00_00_0_01_0063_29132195.txt'

In [10]:
# Récupérer le texte du fichier
text = open(os.path.join(data_path, this_file), 'r', encoding='utf-8').read()
text[:500]

"U.R.S.S. LES JOURNALISTES. RUSSES PREPARENT UNE REVOLUTION DE LA PRESSE OFFICIELLE Les rédacteurs en chef des vingt-trois principaux journaux soviétiques ont été convoqués à Moscou la semaine dernière pour participer à une confé rence sur l'amélioration de la presse en U.R.S.S. L'ordre du jour de la conférence était secret et les journaux n'ont publié aucun compte rendu de ses travaux. L’importance de cette réunion était cependant attestée par la présence de Mme Fourtseva, membre du Praesi dium "

In [11]:
# Extraire les mots clés de ce texte
keywords = kw_extractor.extract_keywords(text)
keywords

[('PRESSE', 0.024222911366124002),
 ('journaux', 0.048466587604846074),
 ('presse soviétique', 0.049043047425175505),
 ('PRESSE INDIENNE', 0.06301346187119425),
 ('PRESSE OFFICIELLE', 0.06635017550139964),
 ('PREPARENT UNE REVOLUTION', 0.07064427654248945),
 ('presse soviétique actuelle', 0.092792736075077),
 ('RUSSES', 0.09717784917912838),
 ('principaux journaux soviétiques', 0.10245433883722045),
 ('rédacteurs en chef', 0.1079571191952993),
 ('RUSSES PREPARENT', 0.11092456526686227),
 ('Mme Fourtseva', 0.11526194171429652),
 ('Baulin', 0.12018922527534606),
 ('articles', 0.13363210502971745),
 ('chef', 0.13812884838907058),
 ('articles de propagande', 0.13951003955336688),
 ('journaux soviétiques', 0.14299826436244917),
 ("PRESSE INDIENNE L'Office", 0.14972660226786894),
 ('journaux russes', 0.15190997668400977),
 ('Moscou', 0.15980231490360491),
 ('quotidiens', 0.1664857984594843),
 ('vingt-trois principaux journaux', 0.17110251425180872),
 ('périodiques', 0.17610894164166838),
 ('

In [12]:
# Ne garder que les bigrammes
kept = []
for kw, score in keywords:
    words = kw.split()
    if len(words) == 2:
        kept.append(kw)
kept

['presse soviétique',
 'PRESSE INDIENNE',
 'PRESSE OFFICIELLE',
 'RUSSES PREPARENT',
 'Mme Fourtseva',
 'journaux soviétiques',
 'journaux russes',
 'presse modernes',
 "presse n'est",
 'Presse manquait',
 'principaux journaux',
 "journaux n'ont",
 'Press Registrar',
 'Praesi dium',
 'vingt-trois rédacteurs',
 'villes russes',
 'Alexandre Baulin',
 'journaux occidental',
 'soviétique actuelle',
 'semaine dernière',
 'dirigeants russes',
 'vingt-trois principaux',
 'propagande publiés',
 'grands quotidiens',
 'chef réunis',
 "n'ont publié",
 'compte rendu',
 "INDIENNE L'Office",
 'grand nombre',
 'grandes villes',
 'nouveaux périodiques',
 'périodiques indiens',
 'communiqués officiels',
 'officiels qu’ils',
 'théori cien',
 'journal officiel',
 'grande place',
 'tirage total',
 'sévèrement attaqué',
 'point spécialisés',
 'qu’ils finissent',
 'ressembler davantage',
 'bulletins ministériels',
 "ministériels qu'à",
 'fois pompeux',
 'simples reproductions',
 "rabâchent d'ailleurs",
 'vé

In [13]:
for f in sorted(files)[:10]:
    text = open(os.path.join(data_path, f), 'r', encoding="utf-8").read()
    keywords = kw_extractor.extract_keywords(text)
    kept = []
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 2:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

BE-KBR00_15463334_19590401_22_00_00_0_01_0063_29132195.txt mentions these keywords: presse soviétique, PRESSE INDIENNE, PRESSE OFFICIELLE, RUSSES PREPARENT, Mme Fourtseva, journaux soviétiques, journaux russes, presse modernes, presse n'est, Presse manquait, principaux journaux, journaux n'ont, Press Registrar, Praesi dium, vingt-trois rédacteurs, villes russes, Alexandre Baulin, journaux occidental, soviétique actuelle, semaine dernière, dirigeants russes, vingt-trois principaux, propagande publiés, grands quotidiens, chef réunis, n'ont publié, compte rendu, INDIENNE L'Office, grand nombre, grandes villes, nouveaux périodiques, périodiques indiens, communiqués officiels, officiels qu’ils, théori cien, journal officiel, grande place, tirage total, sévèrement attaqué, point spécialisés, qu’ils finissent, ressembler davantage, bulletins ministériels, ministériels qu'à, fois pompeux, simples reproductions, rabâchent d'ailleurs, vérités élémentaires, élémentaires connues, vulgarisation sci

In [26]:
data_path = "../../data/txt/"

In [27]:
DECADE = '1915'

In [28]:
files = [f for f in sorted(os.listdir(data_path)) if f"_{DECADE[:-1]}" in f]

In [29]:
# Exemple de fichiers
files[:700]

['KB_JB421_1910-10-10_01-00001.txt',
 'KB_JB421_1911-07-31_01-00002.txt',
 'KB_JB421_1913-02-22_01-00001.txt',
 'KB_JB421_1913-05-02_01-00002.txt',
 'KB_JB421_1913-11-23_01-00001.txt',
 'KB_JB421_1914-03-14_01-00001.txt',
 'KB_JB421_1914-03-28_01-00001.txt',
 'KB_JB421_1914-05-13_01-00001.txt',
 'KB_JB421_1914-06-13_01-00001.txt',
 'KB_JB555A_1915-10-21_01-00001.txt',
 'KB_JB555A_1917-07-31_01-00002.txt',
 'KB_JB555A_1917-08-31_01-00001.txt',
 'KB_JB555A_1917-09-19_01-00002.txt',
 'KB_JB555A_1917-12-12_01-00001.txt',
 'KB_JB555A_1918-11-02_01-00001.txt',
 'KB_JB555A_1918-11-16_01-00001.txt',
 'KB_JB555_1910-10-09_01-00001.txt',
 'KB_JB555_1910-10-12_01-00004.txt',
 'KB_JB555_1911-04-22_01-00001.txt',
 'KB_JB555_1911-10-30_01-00001.txt',
 'KB_JB555_1911-11-17_01-00001.txt',
 'KB_JB555_1911-12-02_01-00001.txt',
 'KB_JB555_1911-12-05_01-00001.txt',
 'KB_JB555_1911-12-07_01-00004.txt',
 'KB_JB555_1912-01-16_01-00001.txt',
 'KB_JB555_1912-01-21_01-00004.txt',
 'KB_JB555_1912-01-30_01-00004.

In [30]:
texts = [open(data_path + f, "r", encoding="utf-8").read() for f in files]

In [31]:
def preprocessing(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

In [32]:
N_CLUSTERS = 5

In [33]:
km_model = KMeans(n_clusters=N_CLUSTERS)

In [35]:
clustering = collections.defaultdict(list)

for idx, label in enumerate(clusters):
    clustering[label].append(files[idx])

NameError: name 'clusters' is not defined

In [36]:
pprint(dict(clustering))

{}


In [37]:
pprint(dict(clustering))

{}


In [38]:
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(tfidf_vectors.toarray())

NameError: name 'tfidf_vectors' is not defined

In [ ]:
reduced_vectors[:500]

In [ ]:
x_axis = reduced_vectors[:, 0]
y_axis = reduced_vectors[:, 1]

plt.figure(figsize=(10,10))
scatter = plt.scatter(x_axis, y_axis, s=100, c=clusters)

# Ajouter les centroïdes
centroids = pca.transform(km_model.cluster_centers_)
plt.scatter(centroids[:, 0], centroids[:, 1],  marker = "x", s=100, linewidths = 2, color='black')

# Ajouter la légende
plt.legend(handles=scatter.legend_elements()[0], labels=set(clusters), title="Clusters")